# Orr-Sommerfeld equation

The Orr-Sommerfeld equation is an eigenvalue problem that determines whether a parallel fluid flow profile $u = (u(z), 0, 0)$ is unstable, given the velocity profile and the Reynolds number. This is an important first step for studying whether a fluid flow, such as the flow in a pipe, is likely to remain laminar or to transition to turbulence.

In [ ]:
from functools import lru_cache
import numpy as np
from numpy import pi
from numpy import sqrt, sin, cos, tan, sinh, cosh, exp, dot
# from numpy.linalg import eig, eigh
from scipy.linalg import eig, eigh, eigvals
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from differentials import *

In [ ]:
def discretise(xa, xb, nx=101): # , bcs="periodic"):
    """Returns the xs, the spacing dx between the xs,
    and the identity and first and second derivative
    matrices, computed using a 2nd-order scheme."""
    xs = np.linspace(xa, xb, nx)
    dx = xs[1] - xs[0]

    eye = np.eye(nx)
    
    # Use central finite differences
    ddx = (np.roll(eye, 1, 1) - np.roll(eye, -1, 1)) / (2*dx)
    d2dx2 = (np.roll(eye, 1, 1) - 2*eye + np.roll(eye, -1, 1)) / (dx*dx)
    # Use one-way differences at the boundaries
    ddx[0, 4:] = 0; ddx[0, :4] = (-11/6, 3, -3/2, 1/3) / dx
    ddx[-1, :-4] = 0; ddx[-1, -4:] = (-1/3, 3/2, -3, 11/6) / dx
    d2dx2[0, 4:] = 0; d2dx2[0, :4] = (2, -5, 4, -1) / (dx*dx)
    d2dx2[-1, :-4] = 0; d2dx2[-1, -4:] = (-1, 4, -5, 2) / (dx*dx)
    return xs, dx, eye, ddx, d2dx2



def myeig(op, eye, *args, **kwargs):
    nx = op.shape[0]
    eigvals, eigvecs = eig(op, eye, *args, **kwargs)
    ordering = sorted(range(nx), key=lambda i: np.real(eigvals)[i])
    eigvals = eigvals[ordering]
    eigvecs = eigvecs[:, ordering] * sqrt(nx)
    return eigvals, eigvecs

In [ ]:
xa = -1; xb = 1; nx = 201;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
d4dx4 = d4dx4_mat(nx, dx)

# Rey = 5772
# alpha = 1.02
Us = (1-xs**2)
Upps = -2 + np.zeros(Us.shape)
# Us = xs
# Upps = np.zeros(Us.shape)

def augment(mat, pad):
    """Return a padded matrix."""
    size = mat.shape[0]
    return np.block([
        [np.zeros([pad, pad]), np.zeros([pad, size]), np.zeros([pad, pad])],
        [np.zeros([size, pad]), mat, np.zeros([size, pad])],
        [np.zeros([pad, pad]), np.zeros([pad, size]), np.zeros([pad, pad])],
    ])

@lru_cache(maxsize=None)
def orrsommsolver(Rey, alpha, *args, **kwargs):
    orrsomm_left = 1j / (alpha * Rey) * (d4dx4 - 2*(alpha**2) * d2dx2 + (alpha**4) * eye) \
                   - np.diag(Upps) \
                   + np.diag(Us) @ (d2dx2 - (alpha**2)*eye)
    orrsomm_right = (d2dx2 - (alpha**2)*eye)

    # Boundary conditions
    orrsomm_left = augment(orrsomm_left, 2)
    orrsomm_right = augment(orrsomm_right, 2)
    orrsomm_left[[0, 1, -2, -1], :] = 0
    orrsomm_right[0, :] = 0; 
    orrsomm_right[0, 2] = 1
    orrsomm_right[-1, :] = 0; 
    orrsomm_right[-1, -3] = 1
    orrsomm_right[1, :] = 0; 
    orrsomm_right[1, 2:5] = (3/2, -2, 1/2) / dx;
    orrsomm_right[-2, :] = 0; 
    orrsomm_right[-2, -5:-2] = (-1/2, 2, -3/2) / dx;
    
#     print(orrsomm_left)
#     print(orrsomm_right)
    
    eigvals, eigvecs = myeig(orrsomm_left, orrsomm_right, *args, **kwargs)
    growths = -1j * alpha * eigvals
    
    # Sort those that remain in order of growth size (which is the
    # imaginary part of the eigenvalue, times the wavenumber).
    # Drop the four eigenvalues that came from imposing the BCs.
    keep = sorted(range(len(eigvals)),
                  key=lambda i: abs(eigvals[i]))[6:]
    growths = growths[keep]
    eigvals = eigvals[keep]
    eigvecs = eigvecs[:, keep]    
    
    ordering = sorted(range(len(eigvals)),
                      key=lambda i: np.real(growths[i]), 
                      reverse=True)
    # sort by the growth rate
    growths = growths[ordering]
    eigvals = eigvals[ordering]
    eigvecs = eigvecs[:, ordering]
    eigvecs = eigvecs[2:-2, :]
    return growths, eigvals, eigvecs

In [ ]:
@interact(Rey=widgets.FloatSlider(min=500, max=10000, step=100, 
                                  value=5772, continuous_update=False,
                                  layout=Layout(width="500px")),
          alpha=widgets.FloatSlider(min=0, max=4, step=0.01, 
                                    value=1.02, continuous_update=False,
                                    layout=Layout(width="500px")))
def orrsommsolver_int(Rey, alpha, *args, **kwargs):
    growths, eigvals, eigvecs = orrsommsolver(Rey, alpha, *args, **kwargs)
    ax = plt.gca()
    ax.plot(np.real(growths), np.imag(growths), 'k+')
    ax.grid()
    ax.set_xlim([-1, 0.1])
    ax.set_ylim([-1, -0.2])
    ax.set_title(f"wavespeed = %.3f\ngrowth rate = %.3f" % (
                 np.real(eigvals[0]), np.real(growths[0])))

    plt.show()

In [ ]:
Rey = 5772
alpha = 1.02
growths, eigvals, eigvecs = orrsommsolver(Rey, alpha)
ax = plt.gca()
ax.plot(np.real(growths), np.imag(growths), 'k+')
ax.grid()
ax.set_xlim([-1, 0.1])
ax.set_ylim([-1, -0.2])
ax.set_title(f"wavespeed = %.3f\ngrowth rate = %.3f" % (
                 np.real(eigvals[0]), np.real(growths[0])))

plt.show()

In [ ]:
eigvals

In [ ]:
ax = plt.gca()
ax.plot(np.real(growths), np.imag(growths), 'k+')
ax.grid()
ax.set_xlim([-1, 0.1])
# ax.set_ylim([-1, -0.2])
ax.set_title(f"wavespeed = %.3f\ngrowth rate = %.3f" % (
                 np.real(eigvals[0]), np.real(growths[0])))

plt.show()

In [ ]:
plt.plot(eigvecs[:, 0], xs); 
plt.gca().grid();

## Dispersion relation

In [ ]:
# @lru_cache(maxsize=None)
def maxgrowth(Rey, alpha, *args, **kwargs):
    return np.real(orrsommsolver(Rey, alpha, *args, **kwargs)[0][0])


@interact(Rey=widgets.FloatSlider(min=500, max=10000, step=100, 
                                  value=5772, continuous_update=False,
                                  layout=Layout(width="500px")))
@lru_cache(maxsize=128)
def os_dispersion(Rey): #, *args, **kwargs):
    alphas = np.linspace(0.1, 1.3, 49)
    growths = [maxgrowth(Rey, alpha) #, *args, **kwargs)
               for alpha in alphas]
    ax = plt.gca()
    ax.plot(alphas, growths)
    ax.grid()
    ax.set_title(f"Rey = {Rey}")
    ax.set_xlim([0, 1.4])
    plt.show()